In [70]:
!pip install git+https://github.com/openai/CLIP.git
!wget https://people.eecs.berkeley.edu/~efros/img/Efros__photo_Peter_Badge_crop.jpg
!mv Efros__photo_Peter_Badge_crop.jpg efros.jpg

--2021-09-16 22:05:08--  https://people.eecs.berkeley.edu/~efros/img/Efros__photo_Peter_Badge_crop.jpg
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456031 (445K) [image/jpeg]
Saving to: ‘Efros__photo_Peter_Badge_crop.jpg’

Efros__photo_Peter_ 100%[===================>] 445.34K  1.75MB/s    in 0.2s    

2021-09-16 22:05:08 (1.75 MB/s) - ‘Efros__photo_Peter_Badge_crop.jpg’ saved [456031/456031]



In [49]:
%load_ext autoreload
%autoreload 2
from __future__ import annotations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import copy
import os
from time import perf_counter
import click
import imageio
import numpy as np
import PIL.Image
import torch
import torch.nn.functional as F
import einops

import dnnlib
import legacy

from projector import *

In [51]:
# network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"
network_pkl = "https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-car-config-f.pkl"
target_fname = "efros.jpg"
outdir = "out"
save_video = True
seed = 202
np.random.seed(seed)
torch.manual_seed(seed)
num_steps = 200
device = torch.device('cuda')

In [52]:
gen = Generator(network_pkl, device=device)
latent = gen.sample_latent(10)
images = gen.latent_to_image(latent)
print(latent.shape, images.shape)

torch.Size([10, 1, 512]) torch.Size([10, 3, 512, 512])


In [59]:
gen = Generator(network_pkl, latent_space='w', device=device)
prior = Prior(gen, device=device, prior_type='l2', regularize_w_l2=0.05)

blue_target = torch.cat((torch.zeros(2, 512, 512), 255*torch.ones(1, 512, 512)), 0)
task = Task(device=device, target=blue_target)
projector = Projector(gen, task, prior=prior, device=device)

# Optimize projection.
start_time = perf_counter()

# later TODO mem optimization -> mixed precision, gradient checkpointing, multiGPU 
projected_w_steps = projector.project(
    num_images=6,
    num_steps=100,
)

print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

Computing W midpoint and stddev using 10000 samples...
step    1/100: loss 28893.53
step    2/100: loss 27314.04
step    3/100: loss 23346.91
step    4/100: loss 17142.20
step    5/100: loss 11136.33
step    6/100: loss 7090.74
step    7/100: loss 6215.59
step    8/100: loss 6988.26
step    9/100: loss 8380.51
step   10/100: loss 9475.97
step   11/100: loss 9707.96
step   12/100: loss 9009.28
step   13/100: loss 7402.72
step   14/100: loss 5434.17
step   15/100: loss 3617.15
step   16/100: loss 2286.16
step   17/100: loss 1649.32
step   18/100: loss 1473.41
step   19/100: loss 1535.36
step   20/100: loss 1764.03
step   21/100: loss 2015.69
step   22/100: loss 2162.27
step   23/100: loss 2190.57
step   24/100: loss 2100.80
step   25/100: loss 1930.65
step   26/100: loss 1729.09
step   27/100: loss 1484.46
step   28/100: loss 1168.09
step   29/100: loss 822.96
step   30/100: loss 527.54
step   31/100: loss 330.16
step   32/100: loss 245.18
step   33/100: loss 267.82
step   34/100: loss 3

In [48]:
print(projected_w_steps.shape)
print(projected_w_steps[0,2,0] == projected_w_steps[0,2,1])

torch.Size([100, 6, 16, 512])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        Tr

In [6]:
gen = Generator(network_pkl, device=device)
prior = Prior(gen, device=device, prior_type='cluster', regularize_cluster_weight=0.05)

blue_target = torch.cat((torch.zeros(2, 512, 512), 255*torch.ones(1, 512, 512)), 0)
task = Task(device=device, target=blue_target)
projector = Projector(gen, task, prior=prior, device=device)

# Optimize projection.
start_time = perf_counter()

# later TODO mem optimization -> mixed precision, gradient checkpointing, multiGPU 
projected_w_steps = projector.project(
    num_images=6,
    num_steps=100,
)

print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

Computing W midpoint and stddev using 10000 samples...
step    1/100: loss 19501.38
step    2/100: loss 13564.70
step    3/100: loss 11392.08
step    4/100: loss 8324.23
step    5/100: loss 6648.95
step    6/100: loss 5437.71
step    7/100: loss 4410.93
step    8/100: loss 3269.49
step    9/100: loss 2947.05
step   10/100: loss 3288.89
step   11/100: loss 3944.55
step   12/100: loss 4612.66
step   13/100: loss 4648.57
step   14/100: loss 4356.61
step   15/100: loss 3885.59
step   16/100: loss 3181.38
step   17/100: loss 2515.08
step   18/100: loss 1891.19
step   19/100: loss 1254.38
step   20/100: loss 838.05
step   21/100: loss 633.57
step   22/100: loss 556.24
step   23/100: loss 560.92
step   24/100: loss 540.19
step   25/100: loss 457.61
step   26/100: loss 342.23
step   27/100: loss 230.10
step   28/100: loss 172.67
step   29/100: loss 183.60
step   30/100: loss 223.20
step   31/100: loss 240.28
step   32/100: loss 254.94
step   33/100: loss 253.92
step   34/100: loss 208.81
step 

In [41]:
gen = Generator(network_pkl, device=device)
task = Task(device=device, task_type = 'clip_text', target_str = 'racecar')
prior = Prior(gen, device=device, prior_type='cluster', regularize_cluster_weight=0.01)
projector = Projector(gen, task, prior=prior, device=device)

# Optimize projection.
start_time = perf_counter()

# later TODO mem optimization -> mixed precision, gradient checkpointing, multiGPU 
projected_w_steps = projector.project(
    num_images=12,
    num_steps=200,
)

print (f'Elapsed: {(perf_counter()-start_time):.1f} s')

running k-means on cuda..


[running kmeans]: 99it [00:14,  6.80it/s, center_shift=0.000000, iteration=99, tol=0.000100]


Computing W midpoint and stddev using 10000 samples...
step    1/200: loss 37790.60
step    2/200: loss 36154.43
step    3/200: loss 34212.82
step    4/200: loss 30604.17
step    5/200: loss 25753.32
step    6/200: loss 20279.76
step    7/200: loss 14668.55
step    8/200: loss 9365.54
step    9/200: loss 5350.07
step   10/200: loss 3475.14
step   11/200: loss 4059.57
step   12/200: loss 6345.83
step   13/200: loss 9018.62
step   14/200: loss 10828.66
step   15/200: loss 11665.45
step   16/200: loss 11423.28
step   17/200: loss 10243.14
step   18/200: loss 8322.96
step   19/200: loss 6169.29
step   20/200: loss 4194.33
step   21/200: loss 2706.40
step   22/200: loss 1740.33
step   23/200: loss 1287.90
step   24/200: loss 1293.53
step   25/200: loss 1628.72
step   26/200: loss 2115.52
step   27/200: loss 2539.29
step   28/200: loss 2775.60
step   29/200: loss 2784.66
step   30/200: loss 2600.95
step   31/200: loss 2299.41
step   32/200: loss 1953.39
step   33/200: loss 1618.75
step   34/

In [42]:
# Render debug output: optional video and projected image and W vector.
num_rows = 4
os.makedirs(outdir, exist_ok=True)
with torch.no_grad():
    if save_video:
        video = imageio.get_writer(f'{outdir}/test.mp4', mode='I', fps=10, codec='libx264', bitrate='1M')
        print (f'Saving optimization progress video "{outdir}/test.mp4"')
        for projected_w in projected_w_steps:
            synth_image = projector.gen.latent_to_image(projected_w)
            synth_image = (synth_image + 1) * (255/2)
            synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8).cpu().numpy()
            grid_image = einops.rearrange(synth_image, "(n1 n2) h w c-> (n1 h) (n2 w) c", n1=num_rows)
            video.append_data(grid_image)
            # video.append_data(np.concatenate([target_uint8, synth_image], axis=1))
        video.close()

Saving optimization progress video "out/test.mp4"


running k-means on cuda..























































































































[running kmeans]: 58it [00:10,  5.58it/s, center_shift=0.000000, iteration=58, tol=0.000100]


Computing W midpoint and stddev using 10000 samples...


[running kmeans]: 9it [05:51, 39.06s/it, center_shift=2127.009277, iteration=9, tol=0.000100]
[running kmeans]: 71it [04:15,  3.60s/it, center_shift=416.422058, iteration=71, tol=0.000100]


torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    1/100: loss 24641.39
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    2/100: loss 23522.60
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    3/100: loss 19669.75
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    4/100: loss 13434.20
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    5/100: loss 8015.63
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    6/100: loss 5750.13
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    7/100: loss 6652.37
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    8/100: loss 8021.27
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step    9/100: loss 8385.34
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step   10/100: loss 7734.00
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step   11/100: loss 6643.91
torch.Size([10, 1, 512])
torch.Size([6, 10, 1, 512])
step   12/100: loss 5863.29
torch.Size([10, 1, 512])

In [ ]:
# Save final projected frame and W vector.
target_pil.save(f'{outdir}/target.png')
projected_w = projected_w_steps[-1]
synth_image = G.synthesis(projected_w.unsqueeze(0), noise_mode='const')
synth_image = (synth_image + 1) * (255/2)
synth_image = synth_image.permute(0, 2, 3, 1).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()
PIL.Image.fromarray(synth_image, 'RGB').save(f'{outdir}/projblue.png')
np.savez(f'{outdir}/projected_wblue.npz', w=projected_w.unsqueeze(0).cpu().numpy())

torch.Size([1, 512])
Computing W midpoint and stddev using 10000 samples...
step    1/100: loss 7803.90
step    2/100: loss 9929.82
step    3/100: loss 7444.09
step    4/100: loss 4298.93
step    5/100: loss 2779.84
step    6/100: loss 2723.05
step    7/100: loss 3702.09
step    8/100: loss 4738.89
step    9/100: loss 5457.20
step   10/100: loss 5363.79
step   11/100: loss 4496.58
step   12/100: loss 3311.87
step   13/100: loss 2301.44
step   14/100: loss 1559.28
step   15/100: loss 1139.34
step   16/100: loss 959.92
step   17/100: loss 1137.18
step   18/100: loss 1464.91
step   19/100: loss 1624.43
step   20/100: loss 1567.80
step   21/100: loss 1293.72
step   22/100: loss 914.23
step   23/100: loss 657.61
step   24/100: loss 563.56
step   25/100: loss 528.72
step   26/100: loss 485.11
step   27/100: loss 472.07
step   28/100: loss 468.77
step   29/100: loss 487.82
step   30/100: loss 499.33
step   31/100: loss 453.42
step   32/100: loss 373.87
step   33/100: loss 274.30
step   34/100

In [ ]:
from IPython.display import Video

Video("out/projblue_clusterreg5.mp4")